In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf   
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array, load_img
from tensorflow.keras.applications import ResNet50,Xception
from tensorflow.keras.models import Sequential,Model   
from tensorflow.keras.layers import Dense,Input, GlobalAveragePooling2D, Dense, Dropout, multiply, Reshape, Conv2D, Activation, Add, Dropout, GlobalAveragePooling2D, BatchNormalization, Multiply,MaxPooling2D,Concatenate,Conv2D, UpSampling2D, Cropping2D,Lambda
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, LearningRateScheduler,ModelCheckpoint

import shutil
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import Accuracy


2025-05-12 13:15:34.582473: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747055734.762994      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747055734.816454      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import matplotlib.pyplot as plt    
import matplotlib.image as mpimg

In [3]:
# Set the image size and batch size for training
batch_size =16

In [4]:
# Define input shape 
input_shape = (400,400,3)
    

In [5]:
train_dataset_dir='/kaggle/input/isic-2017-preprocessed-augmented/content/Linear_Exact_Aug/Train'   

In [6]:
Valid_dataset_dir='/kaggle/input/isic-2017-preprocessed-augmented/content/Linear_Exact_Aug/Valid'   

In [7]:
# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,  
)

In [8]:
# Load the training dataset with data augmentation
train_generator = train_datagen.flow_from_directory(
    train_dataset_dir,   
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
)

Found 14018 images belonging to 3 classes.


In [9]:
test_dataset_dir = '/kaggle/input/isic-2017-preprocessed-augmented/content/Linear_Exact_Aug/Test' 

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_data_generator = ImageDataGenerator(
    rescale=1.0/255.0,  # Normalize pixel values to the range [0, 1]
    # Add any other preprocessing options if needed
)

test_generator = test_data_generator.flow_from_directory(
    test_dataset_dir,    
    target_size=(400,400),  # Adjust to match your model's input size
    batch_size=16,           # Adjust batch size as needed   
    class_mode='categorical',  # If you have class labels
    shuffle=False              # Do not shuffle test data
)

Found 600 images belonging to 3 classes.


In [11]:
# Data normalization for validation and testing
val_datagen = ImageDataGenerator(rescale=1.0 / 255.0)   

In [12]:
# Load the validation dataset
val_generator = val_datagen.flow_from_directory(
    Valid_dataset_dir,   
    target_size=input_shape[:2],
    batch_size=batch_size,    
    class_mode='categorical'
)

Found 150 images belonging to 3 classes.


In [13]:
# Create an input layer
input_layer = Input(shape=input_shape)   

In [14]:
import tensorflow_hub as hub    

In [15]:
#bit_l_url = "https://tfhub.dev/google/bit/m-r101x1/1"   

In [17]:
# Load the BiT L model from TensorFlow Hub
#bit_l_model = hub.KerasLayer(bit_l_url, trainable=True)
#bit_l_output = bit_l_model(input_layer)    

ValueError: Exception encountered when calling layer 'keras_layer_1' (type KerasLayer).

A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

Call arguments received by layer 'keras_layer_1' (type KerasLayer):
  • inputs=<KerasTensor shape=(None, 400, 400, 3), dtype=float32, sparse=False, name=keras_tensor>
  • training=None

In [18]:
#bit_l_output_reshaped = Reshape((1, 1, -1))(bit_l_output)  # Reshape to match the expected shape

NameError: name 'bit_l_output' is not defined

In [19]:
# Load the Xception model without top classification layers
xception_base_model = Xception(weights='imagenet', include_top=False, input_shape=input_shape)
for layer in xception_base_model.layers:
    layer.trainable = True     

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [20]:
# Connect the input layer to the base models
xception_features = xception_base_model(input_layer)


In [21]:
# Load the Xception model without top classification layers
resnet50_base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
for layer in xception_base_model.layers:
    layer.trainable = True     


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [22]:
# Connect the input layer to the base models
resnet50_features = resnet50_base_model(input_layer)


CCA Feature Fusion     

In [24]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense, Concatenate

class CCALikeFusion(Layer):
    def __init__(self, latent_dim=100, fusion_type='concat', **kwargs):
        super(CCALikeFusion, self).__init__(**kwargs)
        self.latent_dim = latent_dim
        self.fusion_type = fusion_type

        # Learnable projections
        self.proj_x = Dense(latent_dim, use_bias=False, name="proj_x")
        self.proj_y = Dense(latent_dim, use_bias=False, name="proj_y")

        if self.fusion_type == 'concat':
            self.concat = Concatenate()

    def call(self, inputs):
        x, y = inputs  # x and y are feature maps (e.g., from Xception and ResNet50)

        # Project to latent space
        x_proj = self.proj_x(x)
        y_proj = self.proj_y(y)

        if self.fusion_type == 'concat':
            return self.concat([x_proj, y_proj])
        elif self.fusion_type == 'average':
            return (x_proj + y_proj) / 2.0
        else:
            raise ValueError("fusion_type must be 'concat' or 'average'")

    def get_config(self):
        config = super(CCALikeFusion, self).get_config()
        config.update({
            "latent_dim": self.latent_dim,
            "fusion_type": self.fusion_type
        })
        return config


In [25]:
combined_features = CCALikeFusion(latent_dim=100, fusion_type='concat')([xception_features, resnet50_features])


In [ ]:
#combined_features =multiply([xception_features,bit_l_output_reshaped])   

In [26]:
# Apply Squeeze-and-Excitation block
def se_block(input_tensor):   
    num_channels = input_tensor.shape[-1]
    
    # Squeeze operation (Global Average Pooling)
    squeeze = GlobalAveragePooling2D()(input_tensor)
    squeeze = Reshape((1, 1, num_channels))(squeeze)
    
    # Excitation operation (Fully connected layers)
    excitation = Dense(num_channels // 16, activation='relu')(squeeze)
    excitation = Dense(num_channels, activation='sigmoid')(excitation)
    
    # Scale the input feature maps
    scaled_features = multiply([input_tensor, excitation])
    
    return scaled_features

In [27]:
se_output = se_block(combined_features)
   

In [28]:
# Add a classification head to the combined features
from tensorflow.keras.regularizers import l2
x = tf.keras.layers.GlobalAveragePooling2D()(se_output)
x = tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.1)(x)
x = tf.keras.layers.Dense(2048, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='selu',kernel_regularizer=l2(0.02))(x)
x = BatchNormalization()(x)
output = tf.keras.layers.Dense(3, activation='softmax')(x)  # 7output classes


In [29]:
# Create the ensemble model
model = Model(inputs=input_layer, outputs=output)    

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import plot_model

# Define your model here
model = model

# Specify the file path where you want to save the model architecture image
image_path = 'model_architecture.png'

# Plot the model architecture and save it as an image
plot_model(model, to_file=image_path, show_shapes=True, show_layer_names=True)

In [32]:
model.summary()   

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 400, 400, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ xception (Functional)     │ (None, 13, 13, 2048)   │     20,861,480 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ resnet50 (Functional)     │ (None, 13, 13, 2048)   │     23,587,712 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cca_like_fusion           │ (None, 13, 13, 200)    │        409,600 │ xception[0][0],        │
│ (CCALikeFusion)           │                        │                │ resnet50[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d  │ (None, 200)            │              0 │ cca_like_fusion[0][0]  │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_1 (Reshape)       │ (None, 1, 1, 200)      │              0 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 1, 1, 12)       │          2,412 │ reshape_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 1, 1, 200)      │          2,600 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multiply (Multiply)       │ (None, 13, 13, 200)    │              0 │ cca_like_fusion[0][0], │
│                           │                        │                │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 200)            │              0 │ multiply[0][0]         │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 1024)           │        205,824 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 1024)           │              0 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 2048)           │      2,099,200 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 512)            │      1,049,088 │ dense_3[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 512)            │          2,048 │ dense_4[0][0]          │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 3)              │          1,539 │ batch_normalization_4… │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 48,221,503 (183.95 MB)

 Trainable params: 48,112,831 (183.54 MB)

 Non-trainable params: 108,672 (424.50 KB)

In [33]:
import tensorflow as tf
from tensorflow.keras.losses import categorical_crossentropy

def combined_loss(y_true, y_pred, alpha=0.2, categorical_weight=0.5):
    # Compute the categorical cross-entropy loss
    cat_loss = categorical_crossentropy(y_true, y_pred)

    # Reshape the inputs to get anchor, positive, and negative examples
    anchor = tf.reshape(y_pred[:, 0], shape=(-1, 1))
    positive = tf.reshape(y_pred[:, 1], shape=(-1, 1))
    negative = tf.reshape(y_pred[:, 2], shape=(-1, 1))

    # Compute the distance between the anchor and the positive
    pos_distance = tf.reduce_sum(tf.square(anchor - positive), axis=1)
    
    # Compute the distance between the anchor and the negative
    neg_distance = tf.reduce_sum(tf.square(anchor - negative), axis=1)

    # Compute the triplet loss
    triplet_basic_loss = pos_distance - neg_distance + alpha
    triplet_loss = tf.reduce_mean(tf.maximum(triplet_basic_loss, 0.0), axis=0)

    # Compute the combined loss
    combined_loss = categorical_weight * cat_loss + (1 - categorical_weight) * triplet_loss

    return combined_loss

# Example usage in a Keras model
model.compile(optimizer='adam', loss=combined_loss, metrics=['accuracy'])


In [34]:
# Implement learning rate scheduling   
lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=5, verbose=1)

In [36]:

# Define the model checkpoint callback to save the best weights
model_checkpoint = ModelCheckpoint('ISIC2017_Classification_contour_ensemble5.weights.h5', monitor='val_accuracy', save_best_only=True, save_weights_only=True, verbose=1)


In [37]:
# Implement early stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=30, verbose=1, restore_best_weights=True)

In [ ]:
# Train the model
epochs =30
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    epochs=epochs,   
    validation_data=val_generator,
    validation_steps=val_generator.samples // 32,
    callbacks=[lr_scheduler,model_checkpoint,early_stopping]
)

Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1747057601.691009     111 service.cc:148] XLA service 0x7b3c200023d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747057601.691835     111 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1747057609.537781     111 cuda_dnn.cc:529] Loaded cuDNN version 90300
E0000 00:00:1747057629.525973     111 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1747057629.754910     111 gpu_timer.cc:

438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 698ms/step - accuracy: 0.4598 - loss: 6.3202
Epoch 1: val_accuracy improved from -inf to 0.39062, saving model to ISIC2017_Classification_contour_ensemble5.weights.h5
438/438 ━━━━━━━━━━━━━━━━━━━━ 464s 723ms/step - accuracy: 0.4598 - loss: 6.3118 - val_accuracy: 0.3906 - val_loss: 0.6688 - learning_rate: 0.0010
Epoch 2/30
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 612ms/step - accuracy: 0.5434 - loss: 0.5535
Epoch 2: val_accuracy improved from 0.39062 to 0.60938, saving model to ISIC2017_Classification_contour_ensemble5.weights.h5
438/438 ━━━━━━━━━━━━━━━━━━━━ 271s 619ms/step - accuracy: 0.5434 - loss: 0.5534 - val_accuracy: 0.6094 - val_loss: 0.6101 - learning_rate: 0.0010
Epoch 3/30
  1/438 ━━━━━━━━━━━━━━━━━━━━ 4:24 605ms/step - accuracy: 0.2500 - loss: 0.6061

/usr/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 3: val_accuracy improved from 0.60938 to 0.68182, saving model to ISIC2017_Classification_contour_ensemble5.weights.h5
438/438 ━━━━━━━━━━━━━━━━━━━━ 9s 20ms/step - accuracy: 0.2500 - loss: 0.6061 - val_accuracy: 0.6818 - val_loss: 0.6053 - learning_rate: 0.0010
Epoch 4/30
250/438 ━━━━━━━━━━━━━━━━━━━━ 1:55 612ms/step - accuracy: 0.5297 - loss: 0.5068

In [ ]:
# Plot the training and validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# Plot the training and validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')   
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
from tensorflow.keras.models import load_model

#Load the saved weights from the output directory in the executed model architecture .  
model.load_weights('/kaggle/working/ISIC2017_Classification_contour_ensemble5.h5')



In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}")    
print(f"Test Accuracy: {test_accuracy}")
    

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
#  the true labels and predicted labels for the test dataset
y_true = test_generator.classes
y_pred = model.predict(test_generator).argmax(axis=1)
   
# Compute the confusion matrix
cm = confusion_matrix(y_true, y_pred)
    
# Plot the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=test_generator.class_indices.keys())
disp.plot(cmap='viridis')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
labels = {0:'melanoma', 1:'nevus', 2:'seborrheic_keratosis'}
    

In [ ]:
import seaborn as sns
from sklearn.metrics import classification_report
cm = confusion_matrix(y_true, y_pred)
cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]   
fig, ax = plt.subplots(figsize=(8,6))  
sns.heatmap(cmn, annot=True, xticklabels=labels.values(), yticklabels=labels.values(),cmap=plt.cm.Blues, fmt='.2f')
plt.ylabel('Actual Classes')
plt.xlabel('Predicted Classes')
plt.show(block=False)    
   
# Generate the classification report
report = classification_report(y_true, y_pred)

print("Confusion Matrix:")
print(cm)    
print(report)     

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
print("Precision: "+ str(precision_score(y_true, y_pred, average='weighted')))
print("Recall: "+ str(recall_score(y_true, y_pred, average='weighted')))
print("Accuracy: " + str(accuracy_score(y_true, y_pred)))
f1 = f1_score(y_true, y_pred, average='weighted')
print("F1 Score: " + str(f1))
     

In [ ]:
print("Precision: "+ str(precision_score(y_true, y_pred, average='macro')))
print("Recall: "+ str(recall_score(y_true, y_pred, average='macro')))
print("Accuracy: " + str(accuracy_score(y_true, y_pred)))
f1 = f1_score(y_true, y_pred, average='macro')
print("F1 Score: " + str(f1))
    

In [ ]:
print("Precision: "+ str(precision_score(y_true, y_pred, average='micro')))
print("Recall: "+ str(recall_score(y_true, y_pred, average='micro')))
print("Accuracy: " + str(accuracy_score(y_true, y_pred)))
f1 = f1_score(y_true, y_pred, average='micro')
print("F1 Score: " + str(f1))
         

In [ ]:
from sklearn.metrics import roc_curve, auc
y_true = test_generator.classes   
# Get the probabilities for each class (0 and 1) from the model predictions
y_prob = model.predict(test_generator)

# Compute the ROC curve for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(train_generator.num_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true == i, y_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
   
# Plot the ROC curves
plt.figure()
for i in range(train_generator.num_classes):
    plt.plot(fpr[i], tpr[i], label=f'Class {i} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score    
print("weighted Roc score: " + str(roc_auc_score(y_true,y_prob,multi_class='ovr',average='weighted')))
print("macro Roc score: " + str(roc_auc_score(y_true,y_prob,multi_class='ovr',average='macro')))    
print("micro Roc score: " + str(roc_auc_score(y_true,y_prob,multi_class='ovr',average='micro')))  